In [93]:
#dependencies
import pandas as pd
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import Session

In [80]:
#read the csv file into a pandas dataframe
bear_attacks_df = pd.read_csv("../Resources/north_america_bear_killings.csv")
bear_attacks_df

,Name,age,gender,Date,Month,Year,Type,Location,Description,Type of bear,Hunter,Grizzly,Hikers,Only one killed
0,Erin Johnson,27,female,2017-06-19,6,2017,Wild,"Pogo mine, Alaska","Johnson, a contract employee for Pogo Mine, wa...",Black bear,0,0,0,1
1,Patrick Cooper,16,male,2017-06-18,6,2017,Wild,"Indian, Alaska",Cooper was chased and mauled by a bear while r...,Black bear,0,0,0,1
2,Daniel Ward O'Connor,27,male,2015-05-10,5,2015,Wild,"near Mackenzie, British Columbia",Ward was killed by a bear while he slept near ...,Black bear,0,0,0,1
3,Darsh Patel,22,male,2014-09-21,9,2014,Wild,"near West Milford, New Jersey",Patel was about to begin hiking with four frie...,Black bear,0,0,1,1
4,Lorna Weafer,36,female,2014-05-07,5,2014,Wild,"near Fort McMurray, Alberta","Weafer, a Suncor worker was attacked at the re...",Black bear,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,Baby Laird,1,NaN,1908-10-05,10,1908,Captive,"Tucson, Arizona",After a bear escaped from a cage at Elysian Gr...,Black bear,0,0,0,1
162,John Dicht,18,male,1906-11-24,11,1906,Wild,"Elk County, Pennsylvania","Thinking the bear was dead, Dicht began skinni...",Black bear,0,0,0,1
163,Mary Porterfield,3,female,1901-05-19,5,1901,Wild,"Job, West Virginia",The children were gathering flowers near their...,Black bear,0,0,0,0
164,Wilie Porterfield,5,male,1901-05-19,5,1901,Wild,"Job, West Virginia",The children were gathering flowers near their...,Black bear,0,0,0,0


In [81]:
#inspect the columns
bear_attacks_df.columns

Index(['Name', ' age', 'gender', 'Date', 'Month', 'Year', 'Type', 'Location',
       'Description', 'Type of bear', 'Hunter', 'Grizzly', 'Hikers',
       'Only one killed'],
      dtype='object')

In [82]:
#age column has a leading space - remove that and make all others lower case
bear_attacks_df = bear_attacks_df.rename(columns={'Name': 'name', ' age': 'age', 'Date': 'date', 'Type': 'type', 'Location': 'location', 'Description': 'description', 'Type of bear': 'bear_type'})


In [83]:
#we only care about attacks in National Parks - filter for those - these will include Canada's National Parks as well as USA National Parks
in_parks = bear_attacks_df['location'].str.contains('National Park', case=True, regex=False)

#select the relevant columns
in_parks_df = bear_attacks_df[in_parks]
in_parks_df = in_parks_df[['name', 'age', 'date', 'type', 'location', 'description', 'bear_type']]


In [84]:
#check for any duplicate records
in_parks_df.shape

(32, 7)

In [85]:
#there are no duplicate records
in_parks_df.drop_duplicates().shape


(32, 7)

In [86]:
#reset the index - this will be the sql primary key
in_parks_df = in_parks_df.reset_index(drop=True)
#give index column name 'id'
in_parks_df.index.name = 'id'

In [87]:
in_parks_df

,name,age,date,type,location,description,bear_type
id,,,,,,,
0,Lance Crosby,63,2015-08-07,Wild,"Yellowstone National Park, Wyoming","Crosby, an employee at a medical clinic in the...",Brown bear
1,Richard White,49,2012-08-24,Wild,"Denali National Park, Alaska",White was backpacking alone along the Toklat R...,Brown bear
2,John Wallace,59,2011-08-24,Wild,"Yellowstone National Park, Wyoming",Wallace's remains were found by hikers on the ...,Brown bear
3,Brian Matayoshi,57,2011-07-06,Wild,"Yellowstone National Park, Wyoming",Matayoshi and his wife were hiking the Wapiti ...,Brown bear
4,Glenda Ann Bradley,50,2000-05-21,Wild,"Great Smoky Mountains National Park, Tennessee",Bradley was attacked and partially consumed by...,Black bear
5,Timothy Treadwell,46,2003-10-05,Wild,"Katmai National Park, Alaska",Treadwell and Huguenard's corpses were found b...,Brown bear
6,Amie Huguenard,37,2003-10-05,Wild,"Katmai National Park, Alaska",Treadwell and Huguenard's corpses were found b...,Brown bear
7,Craig Dahl,26,1998-05-17,Wild,"Glacier National Park, Montana",Dahl's partially consumed remains were found t...,Brown bear
8,Robert Bell,33,1996-08-23,Wild,"Gates of the Arctic National Park, Alaska",Bell was killed while hiking with a friend nea...,Brown bear


In [88]:
##in pg admin create db 'national_parks_db'
## create table 
# CREATE TABLE bear_attacks (
#     id int PRIMARY KEY,
#     name varchar(40) NOT NULL,
#     age int,
#     date date,
#     type varchar(25),
# 	location varchar(100),
# 	description varchar,
# 	bear_type varchar(25)
# );

In [89]:
#connect to postgres db
connection = "postgres:postgres@localhost:5432/national_parks_db"
engine = create_engine(f'postgresql://{connection}')
Inspector = inspect(engine)

In [90]:
#return the table naes
Inspector.get_table_names()

['bear_attacks']

In [92]:
#write data to sql
in_parks_df.to_sql(name='bear_attacks', con=engine, if_exists='append', index=True)

In [96]:
##verify that data has been written to db
conn = engine.connect()
pd.read_sql("select * from public.bear_attacks", conn)

,id,name,age,date,type,location,description,bear_type
0,0,Lance Crosby,63,2015-08-07,Wild,"Yellowstone National Park, Wyoming","Crosby, an employee at a medical clinic in the...",Brown bear
1,1,Richard White,49,2012-08-24,Wild,"Denali National Park, Alaska",White was backpacking alone along the Toklat R...,Brown bear
2,2,John Wallace,59,2011-08-24,Wild,"Yellowstone National Park, Wyoming",Wallace's remains were found by hikers on the ...,Brown bear
3,3,Brian Matayoshi,57,2011-07-06,Wild,"Yellowstone National Park, Wyoming",Matayoshi and his wife were hiking the Wapiti ...,Brown bear
4,4,Glenda Ann Bradley,50,2000-05-21,Wild,"Great Smoky Mountains National Park, Tennessee",Bradley was attacked and partially consumed by...,Black bear
5,5,Timothy Treadwell,46,2003-10-05,Wild,"Katmai National Park, Alaska",Treadwell and Huguenard's corpses were found b...,Brown bear
6,6,Amie Huguenard,37,2003-10-05,Wild,"Katmai National Park, Alaska",Treadwell and Huguenard's corpses were found b...,Brown bear
7,7,Craig Dahl,26,1998-05-17,Wild,"Glacier National Park, Montana",Dahl's partially consumed remains were found t...,Brown bear
8,8,Robert Bell,33,1996-08-23,Wild,"Gates of the Arctic National Park, Alaska",Bell was killed while hiking with a friend nea...,Brown bear
9,9,Christine Courtney,32,1996-07-05,Wild,"Kluane National Park, Yukon",Courtney was killed while hiking on the Slim's...,Brown bear
